# Предсказание биржевых временных рядов

В этом домашнем задании вам предлагается решить две задачи:
1. Построить предсказательную модель для **посуточного** временного ряда стоимости акций одной из компаний из списка [SNP500](https://en.wikipedia.org/wiki/S%26P_500)
2. Построить предсказательную модель для стоимости достаточно высокочастотного временного ряда (500 мс) стоимости акции. Для этого задания вам будет предоставлена более богатая информация (топ 5 заявок на покупку и продажу, объем биржевого стакана, и др.)

# 1. Microsoft daily price

Ваша задача построить предсказательную модель для стоимости акций компании Microsoft и оценить ее качество используя различные метрики.


In [ ]:
import tarfile
import pandas as pd

all_csvs = dict()
# Open the tar.gz file
with tarfile.open('snp500.tar.gz', 'r:gz') as tar:
    # Iterate over each member file in the tar archive
    for member in tar.getmembers():
        # Check if the member is a regular file and has a .csv extension
        if member.isfile() and member.name.endswith('.csv'):
            # Extract the CSV file from the tar archive
            csv_file = tar.extractfile(member)

            # Create a CSV reader object
            df = pd.read_csv(csv_file, delimiter=',')
            all_csvs[member.name.split('/')[1].split('.')[0]] = df

In [ ]:
df = all_csvs['msft']
df.head()

### Описание данных

- Date - дата
- Open - стоимость акции на момент открытия биржи
- High - наивысшая стоимость акции за время работы биржы в этот день
- Low - наименьшая стоимость акции за время работы биржы в этот день
- Volume - общая сумма операций с данной акцией за рабочий день

Стоимость открытия "Open" это цена по которой была куплена конкретная акция в первый раз за текующий рабочий день. Стоимость закрытия "Close" это цена по которой конкретная акция была куплена в последний раз за текущий рабочий день. Поэтому Close price 1986-03-13 может не совпадать с Open price 1986-03-14.

Ваша задача используя признаки Date, Open, High, Low, Volume для текущей даты и любых предшествующих дат предсказать Open цену на "послезавтра". Считайте что сегодня биржа уже закрылась, т.е. вам доступны Open, High, Low, Volume на момент "сегодня", но **завтрашние не доступны**.

In [ ]:
df['y_reg'] = df['Open'].shift(-2)
df['y_clf'] = ((df['Close'] - df['Open'].shift(-2)) < 0).astype(int)
df = df.dropna()

df.head(10)

### Train, validation, test

В качестве test мы будем использовать последний год. В качестве validation и train вы можете использовать любые периоды, но мы предлагаем вам использовать предшествующий год (2022-2023) для validation и предшествующие два года (2020-2022) для train.

In [ ]:
df.index = df.Date

data_train = df.truncate(before='2020-04-16', after='2022-04-16')
data_val = df.truncate(before='2022-04-17', after='2023-04-17')
data_test = df.truncate(before='2023-04-18')

### Идеи для генерации признаков

- Лаговые признаки: значение временного ряда 1-2-3-7-... тактов назад
- Линейная регрессия построенная по последним 3-5-... известным значениям временного ряда
- Скользящее среднее/минимум/максимум/стандартное отклонение/... в окне разной величины (3-5-7-...) с различными сдвигами: 1-2-7-...
- Относительное изменение величины временного ряда по сравнению с 1-2-7-... тактами назад
- Абсолютное изменение величины временного ряда по сравнению с 1-2-7-... тактами назад
- Среднее за те же дни недели за последние 2-3-4-... недель
- Все вышеперечисленное но для дополнительных признаков Low, High, Volume:
- Разность между Low и High ценами вчера
- Логарифм Volume
- Оценки числа транзацкций за рабочий день, сегодня, вчера, позавчера,...
- Разность между известными Close и Open ценами (цена на момент открытия сегодня, по сравнению с ценой на момент закрытия вчера)
- день недели (понедельник, пятница, праздник?)

и так далее.


При желании вы так же можете использовать информацию о стоимости любых других акций из предоставленного архива `snp500` (всего их там 501 штука). Главное при генерации признаков **не заглядывать в будущее**. Примеры "заглядывания в будущее":
- Использовать данные о "завтрашней" цене при предсказании "послезавтрашней". Информация о "завтрашней" цене не доступна вам **сегодня**.
- Использовать данные Volume при попытке предсказать Close цену по Open цене. Значение **Volume** будет доступно только после закрытия биржы, т.е. уже после того как будет зафиксирована Close цена.
- Использовать данные Highe, Low, при попытке предсказать Close цену по Open цене. См. предыдущий пункт.



### 1.1 Предсказание цены

Имплементируйте функцию `generate_features` и обучите три модели регрессии для предсказания цены акции послезавтра `y_reg`:
- Линейная модель с l2 регуляризацией
- Случайный лес
- Градиентный бустинг (если используете catboost не забудьте использовать значение параметра `has_time=True`)

Не забудьте удалить из признаков столбцы `y_reg` и `y_clf` перед обучением моделей. Для подбора оптимальных гиперпараметров используйте вашу любимую библиотеку (sklearn, HyperOpt, Optuna) и validation набор данных. Используйте стандартизацию данных если это необходимо. Финальные метрики считайте используя функцию `get_test_metrics_regression` на тестовом наборе данных.


Помните что при генерации признаков для validation и test данных, если вы используете какие-то признаки из прошлого, то вам потребуется кусочки данных из train/validation чтобы сгенерить признаки для первых наблюдений из validation, test соответственно.


1. Приведите качество трех обученных моделей в терминах mean_absolute_error
2. Изобразите важности признаков моделей. Сравните важности признаков в разных моделях, прокоментируйте сходства и отличия.
3. Изобразите графически предсказания трех моделей на тестовой части данных, прокомментируйте. С чем могут быть связаны особенности предсказания "деревянных" моделей?
4. Измерьте качество модели, которая предсказывает `y_reg` последней доступной ценой акции (Close цена сегодня), сравните с качеством моделей машинного обучения, прокоментируйте.

In [ ]:
def generate_features():
    # TODO
    ...


def print_test_metrics_regression(model, test_features, y_test):
    from sklearn.metrics import mean_absolute_error

    assert test_features.shape[0] == 124, f"Inconsistent number of test samples: {test_features.shape[0]}, should be 124" # необходимо сделать предсказание для каждого примера из test
    try:
    y_pred = model.predict(test_features)
    except Exception as e:
    print("Your model failed with: ", str(e))

    print("MAE:", mean_absolute_error(y_test, y_pred))

In [ ]:
# 1.1.4
from sklearn.metrics import mean_absolute_error

y_pred_last = data_test['Close']
mean_absolute_error(data_test['y_reg'], y_pred_last)

### 1.2 Предсказание роста цены

Вместо того чтобы предсказывать цену, в этом задании вы будете предсказывать факт (строгого) роста цены (послезавтрашней цены открытия по сравнению с сегодняшней ценой закрытия).

Единственное отличие от пункта 1.1 в том что теперь вы предсказываете столбец `y_clf`.

1. Приведите качество в терминах precision и recall, трех обученных моделей. Какой из показателей на ваш взгляд важнее с точки зрения специфики задачи?
2. Для моделей из предыдущего пункта постройте кривые precision и recall в зависимости от порога вероятности, для этого воспользуйтесь функцией `sklearn.metrics.precision_recall_curve`, прокоментируйте.
3. Воспользуйтесь моделями из пункта 1.1, для того чтобы получить предсказание роста цены. Посчитайте метрики precision и recall для предсказаний полученных на основе регрессионных моделей. Сравните качество с классификационными моделями.
4. Измерьте качество модели, которая предсказывает y_clf последним доступным изменением цены акции ($Open - Close$ цена сегодня), сравните с качеством моделей машинного обучения, прокоментируйте.

In [ ]:
# 1.2.5
from sklearn.metrics import precision_score

y_pred_clf = ((data_test['Open'] - data_test['Close']) < 0).astype(int)
precision_score(data_test['y_clf'], y_pred_clf)

## 1.3 Выводы

Прокомментируйте ваши результаты.

1. Сравните важность признаков регрессионных и классификационных моделей, прокомментируйте ваши наблюдения.
2. Что вы можете сказать о точности полученных моделей, пригодны ли они на ваш взгляд для торгов на бирже, почему?


# 2. High frequency trading

### Термины

- **заявка** (order) на покупку или продажу это запрос (на покупку или продажу) какого-то определенного количества ценной бумаги (или дериватива) по определенной цене (котировке).

- **исполнением** (fill) заявки называется исполнение запроса на покупку или продажу. Например, Вася отправил заявку на покупку 100 акций по цене 1 доллар за акцию. А Петя отправил заявку на продажу 100 акций по цене 1 доллар. Петя исполнил заявку Васи.

- **биржевой стакан** (order book) - список всех не исполненных заявок. Заявки в биржевом стакане исполняются в приоритетном порядке, сначала по цене (наилучшая цена в первую очередь), затем по времени (в первую очередь исполняется заявка выставленная раньше).

- **бид** (bid price) - означает выставить заявку на продажу X акций по цене Y.

- **аск** (ask price) - означает выставить заявку на покупку X акций по цене Y.


Величина бид в биржевом стакане всегда меньше чем величина аск (почему?).


![orderbook](https://www.simtrade.fr/blog_simtrade/wp-content/uploads/2023/08/img_Order_book_buy_sell_side_by_side.png")


### Аналогия работы биржи на примере валютных обменников.

Представьте что вам нужно купить наличные доллары, а у вас есть наличные рубли. Вы можете выбрать из 10 обменников. Список из 10 обменников это биржевой стакан. Каждый обменник предлагает вам:
- "цену покупки" (то за сколько рублей вы сможете продать обменнику 1 доллар), это "bid" 
- и "цену продажи" (то за сколько рублей обменник готов у вас купить 1 доллар), это "ask".
"Цена покупки" всегда меньше чем "Цена продажи", это верно даже если сравнивать цены покупки и продажи в разных обменниках (почему?). Количество наличных в конкретном обменнике, это его "объем" (volume = количество акций), в отличие от биржевого стакана обменники не публикуют эту величину. 

Если конкретный обменник готов продать вам только 1000 долларов по наилучшей цене 97.5 рублей за доллар, а вы хотите купить 1500 долларов, то оставшиеся 500 долларов вам придется покупать в другом обменнике (который продает их по 97.6 рублей за доллар). При этом по мере того как у конкретного обменника будет заканчиваться наличные доллары он будет понемногу повышать цену продажи и поднимать цену покупки.



### Описание столбцов данных

- last_price - цена, по которой произошло последнее исполнение заявки.
- mid - (bid1 + ask1) / 2.
- opened_position_qty - сколько заявок на покупку было исполнено за последние 500 мс. Содержит пропуски.
- closed_position_qty - сколько ордеров на продажу было исполнено за последние 500 мс. Содержит пропуски.
- bid1 - наилучшая (наивысшая) цена покупки акции
- bid[2,3,4,5] - [2-я, 3-я, 4-я, 5-я] лучшая (наивысшая) цена покупки
- ask1 - наилучшая (наименьшая) цена продажи
- ask[2,3,4,5] - [2-я, 3-я, 4-я, 5-я] лучшая (наименьшая) цена продажи
- bid1vol - сколько акций в биржевом стакане можно купить по наилучшей цене
- bid[2,3,4,5]vol - сколько акций в биржевом стакане можно купить по [2,3,4,5]-й цене покупки
- ask1vol - сколько акций в биржевом стакане по 1-ой цене продажи?
- ask[2,3,4,5]vol - сколько акций в биржевом стакане по [2,3,4,5]-й цене продажи?
- y - Вырaстет ли средняя цена mid за следующие 2 такта (примерно 1 секунда) в будущее?

Данные упорядочены по времени `id`, но информация о точной дате и времени отсутствует. Время между соседними записями составляет 500 мс (за исключением записей соответствующих концу рабочего дня и началу следующего, но этим фактом предлагается принебречь). В предположении что биржа открыта c 9:30 до 16:00 данные содержат информацию о чуть более двух неделях торговых сессий.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('hft_data_sample.csv')

In [ ]:
# 60 * 60 * 2 = 7200ms в часе

plt.figure(figsize=(12,5))
plt.title('Mid price over time')
plt.plot(df['id'].iloc[::7200], df['mid'].iloc[::7200], label='hourly')
plt.plot(df['id'].iloc[::54000], df['mid'].iloc[::54000], label='daily')
plt.ylabel('price, $')
plt.xlabel('time')
plt.legend();

In [ ]:
time = 0

plt.title('Order book')
plt.bar(df.iloc[time][['bid1','bid2','bid3','bid4','bid5']],
        df.iloc[time][['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']],
       width=0.25, label='bid price')


plt.bar(df.iloc[time][['ask1', 'ask2', 'ask3', 'ask4',
       'ask5']],
        df.iloc[time][['ask1vol', 'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']],
       width=0.25, label='ask price')

plt.vlines(df.iloc[time][['mid']], 0, 14, color='r', ls='--', label='mid')
plt.legend()
plt.xlabel('Price, $')
plt.ylabel('Volume');

### Train, validation, test

В качестве test мы будем использовать последние двое суток (108000 записей). В качестве validation и train вы можете использовать любые периоды, но мы предлагаем вам использовать предшествующие двое суток для validation и все остальное для train.

In [ ]:
data_test = df.iloc[-108000:]
data_validation = df.iloc[-216000:-108000]
data_train = df.iloc[:-216000]

In [ ]:
data_train.shape, data_validation.shape, data_test.shape

### Идеи для генерации признаков

1. Обычные лаговые признаки, которые использовались в предыдущем задании
2. Суммарное число заявок на покупку и продажу исполненных за последние 500мс, 1с, ...
3. Разница между числом заявок на покупки и продажу исполненных за последние 500мс, 1с, ...
4. Спред (разность) между (1,2,3,4,5) ценой покупки и (1,2,3,4,5) ценой продажи, за последние 500мс, 1с, ...
5. Отношение между ценами покупки и продажи
6. Общая сумма выставленных на текущий момент бумаг на покупку/продажу
7. Разность между последней ценой продажи и максимальной ценой продажи
8. Как поменяется midprice если будет куплено/продано X акций

и так далее


### Задание

Постройте модель машинного обучения для предложенной задачи, измерьте ее качество на тестовой выборке и опишите свое решение. Прокомментируйте различия в качестве полученном для High-frequency data и качестве полученном для по-суточных данных.